In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install kaleido
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pyopencl
!{sys.executable} -m pip install nbformat


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal sit

In [13]:

# Second cell - fix imports
import numpy as np
import pandas as pd  # Add pandas import
import plotly.express as px
from plotly.subplots import make_subplots
import os
import plotly.graph_objects as g

In [15]:



def visualize(csv_paths: list[str]):
    '''visualizes cub, futhark and our impl of radix-sort for a list of csv files'''
    
    # Create figure with subplots
    fig = make_subplots(
        rows=len(csv_paths), 
        cols=1,
        subplot_titles=[f'Data type: {path.split("_")[1].split(".")[0]}' for path in csv_paths]
    )
    
    dtype_sizes = {
        'u8': 1,
        'u16': 2,
        'u32': 4,
        'u64': 8
    }
    
    for i, csv_path in enumerate(csv_paths):
        df = pd.read_csv(csv_path)
        
        # futhark outputs milliseconds not microseconds so we need to adjust
        df.loc[df['impl'] == 'futhark', 'microseconds'] *= 1000    
        
        # Calculate GB/s based on data type
        df['GB/s'] = df.apply(
            lambda row: (row['dataset_size'] * dtype_sizes[row['data_type']]) / 
                       (row['microseconds'] * 1e6 * 1e9),
            axis=1
        )
        
        # Create separate traces for each implementation and data type
        for impl in df['impl'].unique():
            for dtype in df['data_type'].unique():
                mask = (df['impl'] == impl) & (df['data_type'] == dtype)
                name = f"{impl} ({dtype})"
                
                fig.add_trace(
                    g.Scatter(
                        x=df[mask]['dataset_size'],
                        y=df[mask]['GB/s'],
                        mode='lines+markers',
                        name=name,
                        legendgroup=name,
                        showlegend=(i == 0)  # Show legend only for first subplot
                    ),
                    row=i+1, 
                    col=1
                )
    
    # Update layout
    fig.update_layout(
        title='Radix Sort Performance Comparison',
        height=300 * len(csv_paths),  # Adjust height based on number of plots
        showlegend=True,
        hovermode='x unified'
    )
    
    # Update axes for each subplot
    for i in range(len(csv_paths)):
        fig.update_xaxes(
            title='Dataset Size',
            type='log',
            row=i+1, 
            col=1
        )
        fig.update_yaxes(
            title='Throughput (GB/s)',
            type='log',
            row=i+1, 
            col=1
        )
    
    return fig
# Update files list to include all benchmark files


def save_fig(fig : g.Figure, name: str):
    current_dir = os.getcwd()
    os.chdir('..') # move from src to root
    try:
        fig.write_image(f'report/images/{name}.png')  # Changed from write_png to write_image
    except Exception as e:
        print(f"Error writing image: {e}")
        os.chdir(current_dir) # move back to src
    os.chdir(current_dir)


files = ['benchmarks_u8.csv', 'benchmarks_u8.csv']

# Main execution now only needs one call since we're creating a single combined figure
fig = visualize(files)
save_fig(fig, 'combined_benchmarks')

for file in files:
    fig = visualize(file)
    save_fig(fig, file.split('_')[1].split('.')[0])


" \nfor file in files:\n    fig = visualize(file)\n    save_fig(fig, file.split('_')[1].split('.')[0]) "